In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.linalg import expm
from scipy.integrate import odeint
from scipy.integrate import ode
from scipy.optimize import fsolve
from tqdm import tqdm
import time

In [6]:
def dagger(matrix):
    return np.transpose(np.conj(matrix))

def commutator(matrix_1, matrix_2, sign):
    if sign == 0:
        return matrix_1 @ matrix_2 - matrix_2 @ matrix_1
    else:
        return matrix_1 @ matrix_2 + matrix_2 @ matrix_1
    
def propagate(state, H_eff, dt):
    U_eff = expm(-1j * H_eff * dt)
    state = U_eff @ state
    dp = 1 - np.linalg.norm(state)
    return state, dp


In [2]:
# For N = 1 , {|g), |e)}
# For N = 2 , {|g)|g), |g)|e), |e)|g), |e)|e)}
def phi_basis_vecs(N):
    phi = np.array([[0+0j]*(2**N)]).transpose()
    base = []
    for i in range(2**N):
        phi[i] += 1
        base.append(phi)
        phi = np.array([[0+0j]*(2**N)]).transpose()
    return base

# For N = 1 , {GG=|g)(g|, GE=|g)(e|, EG=|e)(g|, EE=|e)(e|}
# For N = 2 , {GGGG,GGGE,GGEG,GGEE,GEGG,GEGE,GEEG,GEEE,EGGG,EGGE,EGEG,EGEE,EEGG,EEGE,EEEG,EEEE}
def rho_basis_ops(N):
    zero = np.zeros((2,2), dtype = complex)
    rho = np.zeros((2,2), dtype = complex)
    base = []
    for i in range(N-1):
        rho = np.kron(rho, zero)
    for i in range(2**N):
        for j in range(2**N):
            rho[i][j] = 1
            base.append(rho)
            rho= np.zeros((rho.shape), dtype = complex)
    return base

def id(N):
    matrix = np.array([[1, 0],[0, 1]], dtype = complex)
    identity = np.array([[1, 0],[0, 1]], dtype = complex)
    for i in range(N-1):
        matrix = np.kron(matrix,identity)
    return matrix

def S_x(N, n):
    s_x = np.array([[0, 1], [1, 0]], dtype=complex)
    identity = np.array([[1, 0],[0, 1]], dtype = complex)
    matrix = np.array([1], dtype = complex)
    for i in range(N):
        if i != n-1:
            matrix = np.kron(matrix, identity)
        elif i == n-1:
            matrix = np.kron(matrix, s_x)
    return matrix

def S_plus(N, n):
    s_plus =  np.array([[0, 0], [1, 0]], dtype=complex)
    identity = np.array([[1, 0],[0, 1]], dtype = complex)
    matrix = np.array([1], dtype = complex)
    for i in range(N):
        if i != n-1:
            matrix = np.kron(matrix, identity)
        elif i == n-1:
            matrix = np.kron(matrix, s_plus)
    return matrix

def S_minus(N, n):
    s_minus = np.array([[0, 1], [0, 0]], dtype=complex)
    identity = np.array([[1, 0],[0, 1]], dtype = complex)
    matrix = np.array([1], dtype = complex)
    for i in range(N):
        if i != n-1:
            matrix = np.kron(matrix, identity)
        elif i == n-1:
            matrix = np.kron(matrix, s_minus)
    return matrix

# Higher order integration of wavefunction in Quantum Jump Monte Carlo method


Pseudo-code for higher order integration of wavefunction:

1. Pick a random number r lying in (0,1)
2. Solve $t_{1}$ in the equation $||e^{-iH_{eff}(t_{1}-t)}\ket{\psi(t)}||^{2} = r$ 
3. Propagate the state $\ket{\psi(t)}$ from current time $t$ to the chosen time $t_{1}$ : Apply $e^{-iH_{eff}\delta t}\ket{\psi(t)}$ M times, where M is number of segments for the time interval $(t, t_{1})$, each segment is of length $\delta t$
4. Jump occurs at time $t_{1}$ : Apply $L\ket{\psi(t_{1})}$
5. Update current time $t$ to $t_{1}$
6. Starting from t = 0 , repeat the above 1.~5. until t > t_total

In [7]:
omega = 18
gamma = omega / 3
L = S_minus(1,1)
H = omega * S_x(1,1)
H_eff = H - 1j/2 * gamma * dagger(L) @ L
dt = 0.000001

In [9]:
current_time = 0
state = phi_basis_vecs(1)[0]
print(state)

state_1, dp = propagate(state, H_eff, dt)
print(state)
print(dp)

[[1.+0.j]
 [0.+0.j]]
[[1.+0.j]
 [0.+0.j]]
3.3306690738754696e-16


In [ ]:
# Higher order integration of wavefunction in Quantum Jump Monte Carlo method


t_total = 1.2
U_eff = expm(-1j * H_eff * dt) 


N = 700
N_ne_values = []  
N_state_vecs = []  
for n in tqdm(range(N)):  

    state = phi_basis_vecs(1)[0]
    t_current = 0.0
    
    
    def norm_sqrt(state):
        return np.linalg.norm(state)**2
    def equation(t_next, t_current, H_eff, state, r):
        term = expm(-1j * H_eff * (t_next - t_current)) @ state 
        return np.linalg.norm(term)**2 - r
    
    jump_num = 0
    state_vecs = []
    ne_values = []
    while t_current < total_t: 
        #print("Current time is", t_current) 
        
        # at time t    
        # solve t_next for |exp[-i * H_eff(t_next-t_current)]|phi(t_current)>|^2 = r , where r is random number 0 to 1
        r = np.random.rand()
        r = round(r, 5)
        t_next = fsolve(equation, 0.00001+t_current, args=(t_current, H_eff, state, r))[0]
        t_next = round(t_next, 5)
        #print("Correspond to random number",r, ",the chosen t_next is", t_next) 
        
        if t_next > total_t:
            t_interval = np.arange(t_current, t_total, dt)
            #print("Propagate the wavefunction through the time interval [", t_current,", ",t_total, "], with ",len(t_interval)," time steps") 
        else:
            t_interval = np.arange(t_current, t_next, dt)
            #print("Propagate the wavefunction through the time interval [", t_current,", ",t_next, "], with ",len(t_interval)," time steps") 
    
        # propagate the state by U_eff = expm(-1j * H_eff * dt) from t_current to t_next
        for t in t_interval:
            state = U_eff @ state
            state = state / np.linalg.norm(state)
            state_vecs.append(state)
            ne = np.abs(state[1][0])**2
            ne_values.append(ne) 
    
        # at time = t_next , apply jump operator to the state
        if t_next < t_total:
            #print("Jump occurs at t =", t_next)
            #print("Just before the jump, the state is:", state)
            jump_num += 1
            state = L @ state
            state = state / np.linalg.norm(state)
            #print("Just after the jump, the state is:", state)
            state_vecs.append(state)
            ne = np.abs(state[1][0])**2
            ne_values.append(ne) 
    
        # update time t_current to time t_next
        t_current = t_next
        
    time_values = np.arange(0, t_total, dt)

    N_state_vecs.append(np.array(state_vecs))
    N_ne_values.append(np.array(ne_values))

print("Simulation complete")

In [ ]:
for n in range(N):
    N_ne_values[n] = N_ne_values[n][:len(time_values)]

plt.figure(figsize=(8,6))
plt.plot(time_values, N_ne_values[0], label='ne1(t)')
plt.plot(time_values, N_ne_values[2], label='ne2(t)')
plt.title('time-evolved excitation density')
plt.xlabel('Time')
plt.ylabel('n_e(t)')
plt.legend()
plt.show()

In [ ]:
N_ne_sum = np.zeros((len(N_ne_values[0])), dtype=complex)
for n in range(N):
    N_ne_sum += N_ne_values[n]

N_ne_avg = N_ne_sum / N

plt.figure(figsize=(8,6))
plt.plot(time_values, N_ne_avg, label='ne(t)')

plt.title('time-evolved excitation density')
plt.xlabel('Time')
plt.ylabel('n_e(t)')
plt.legend()
plt.show()

In [3]:
A = [1,2,3,4]
B = [1,2,3,4]
plt.figure(figsize=(8,6))
%%capture captured_plot1
plt.plot(A, B, label='y=x')
plt.title('Fig.1')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

UsageError: Line magic function `%%capture` not found.


<Figure size 800x600 with 0 Axes>